# Reconocimiento de Entidades Nombradas
## Ejercicio Práctico - Procesamiento de Lenguaje Natural

**Objetivos de Aprendizaje:**
1. Implementar NER usando modelos pre-entrenados en español
2. Crear interfaces interactivas con Gradio
3. Comparar enfoques: Transformers vs API Gemini
4. Desarrollar prototipos rápidos para aplicaciones de PLN

---
**Entorno recomendado:** Google Colab o Amazon SageMaker Studio

**Tiempo estimado:** 60-90 minutos

## Instalación de Dependencias

In [1]:
# Instalación de librerías necesarias
%%capture
!pip install -q transformers torch gradio google-genai

In [2]:
# Verificar instalación
import sys
print(f"Python: {sys.version}")
print("Todas las dependencias instaladas correctamente")

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Todas las dependencias instaladas correctamente


## Configuración de APIs

### Para Google Colab:
1. Andá a la barra lateral izquierda y hacé clic en 🔑 (Secrets)
2. Agrega una nueva clave: `GOOGLE_API_KEY`
3. Pega tu API key de Google AI Studio

### Para SageMaker Studio:
1. Configura las variables de entorno en tu instancia
2. O usa el método de input manual más abajo

In [17]:
import os
import warnings
warnings.filterwarnings('ignore')

# Configuración de API Key para Gemini
try:
    # Método 1: Google Colab Secrets
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key cargada desde Google Colab Secrets")
except:
    # Método 2: Variables de entorno (SageMaker)
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        print("API Key cargada desde variables de entorno")
    else:
        print("No se encontró GOOGLE_API_KEY")
        print("Podes continuar solo con la parte de Transformers")

API Key cargada desde Google Colab Secrets


---
# PARTE 1: NER con Transformers de Hugging Face

Utilizaremos un modelo especializado en español que puede identificar:
- **PER**: Personas
- **LOC**: Lugares
- **ORG**: Organizaciones  
- **MISC**: Misceláneo

In [18]:
from transformers import pipeline
import torch

# Verificar disponibilidad de GPU
device = 0 if torch.cuda.is_available() else -1
print(f"🖥️  Dispositivo: {'GPU' if device == 0 else 'CPU'}")

# Cargar modelo de NER en español
print("📥 Cargando modelo de NER para español...")
MODEL_NAME = "mrm8488/bert-spanish-cased-finetuned-ner"

try:
    ner_pipeline = pipeline(
        "ner",
        model=MODEL_NAME,
        aggregation_strategy="simple",
        device=device
    )
    print(f"Modelo {MODEL_NAME} cargado exitosamente")
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")
    ner_pipeline = None

🖥️  Dispositivo: CPU
📥 Cargando modelo de NER para español...


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Modelo mrm8488/bert-spanish-cased-finetuned-ner cargado exitosamente


In [19]:
# Texto de ejemplo con contexto argentino
texto_ejemplo = """
Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.
"""

In [20]:
def analizar_entidades_transformers(texto):
    """Procesa texto y extrae entidades usando Transformers"""
    if not ner_pipeline:
        return []

    entidades = ner_pipeline(texto)

    # Formatear resultados
    resultados = []
    for ent in entidades:
        resultados.append({
            'texto': ent['word'],
            'etiqueta': ent['entity_group'],
            'confianza': round(ent['score'], 3),
            'posicion': (ent['start'], ent['end'])
        })

    return resultados

In [21]:
# Probar el modelo
print("🔍 Analizando texto de ejemplo...")
print(f"📝 Texto: {texto_ejemplo.strip()}")
print("\n📊 Entidades encontradas:")

entidades_encontradas = analizar_entidades_transformers(texto_ejemplo)
for ent in entidades_encontradas:
    print(f"  • {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔍 Analizando texto de ejemplo...
📝 Texto: Hola, soy María González y trabajo en la Universidad de Buenos Aires.
Vivo en el barrio de San Telmo y mi empresa favorita es MercadoLibre.
La semana pasada visité el Obelisco con mi amigo Carlos Pérez,
quien trabaja en Google Argentina. Nos encontramos en la estación
Constitución del subte y fuimos a comer un asado en La Boca.

📊 Entidades encontradas:
  • María González → PER (confianza: 0.9990000128746033)
  • Universidad de Buenos Aires → ORG (confianza: 0.9990000128746033)
  • San Telmo → LOC (confianza: 0.9980000257492065)
  • MercadoLibre → ORG (confianza: 0.996999979019165)
  • Obelisco → LOC (confianza: 0.9959999918937683)
  • Carlos Pérez → PER (confianza: 1.0)
  • Google Argentina → ORG (confianza: 0.9860000014305115)
  • Constitución → LOC (confianza: 0.9950000047683716)
  • La Boca → LOC (confianza: 1.0)


---
# PARTE 2: NER con API de Gemini

Utilizaremos la API de Gemini para un análisis más detallado y contextual.

In [22]:
# Configurar cliente Gemini
cliente_gemini = None

if GOOGLE_API_KEY:
    try:
        from google import genai
        cliente_gemini = genai.Client(api_key=GOOGLE_API_KEY)
        print("Cliente Gemini configurado correctamente")
    except Exception as e:
        print(f"Error al configurar Gemini: {e}")
else:
    print("API Key de Gemini no disponible")
    print("Podes obtener una gratis en: https://ai.google.dev/")

Cliente Gemini configurado correctamente


In [23]:
def analizar_entidades_gemini(texto):
    """Analiza entidades usando Gemini API"""
    if not cliente_gemini:
        return "❌ Cliente Gemini no disponible"

    prompt = f"""
    Extraé todas las entidades nombradas del siguiente texto en español argentino y clasificálas:

    CATEGORÍAS:
    - PERSONA: Nombres de personas
    - LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
    - ORGANIZACIÓN: Empresas, universidades, instituciones
    - MISCELÁNEO: Otros nombres propios (productos, eventos, marcas)

    FORMATO DE RESPUESTA:
    [ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

    TEXTO A ANALIZAR:
    {texto}
    """

    try:
        respuesta = cliente_gemini.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt]
        )
        return respuesta.text
    except Exception as e:
        return f"❌ Error: {e}"

# Probar Gemini si está disponible
if cliente_gemini:
    print("🔍 Analizando con Gemini...")
    resultado_gemini = analizar_entidades_gemini(texto_ejemplo)
    print("\nAnálisis de Gemini:")
    print(resultado_gemini)
else:
    print("⏭️  Saltando análisis con Gemini (API Key no disponible)")

🔍 Analizando con Gemini...

Análisis de Gemini:
Aquí están las entidades nombradas extraídas del texto, clasificadas y explicadas:

*   María González → PERSONA → Nombre de la persona que habla.
*   Universidad de Buenos Aires → ORGANIZACIÓN → Nombre de una universidad.
*   San Telmo → LUGAR → Nombre de un barrio.
*   MercadoLibre → ORGANIZACIÓN → Nombre de una empresa.
*   Obelisco → LUGAR → Nombre de un monumento/lugar específico.
*   Carlos Pérez → PERSONA → Nombre del amigo.
*   Google Argentina → ORGANIZACIÓN → Nombre de una empresa (filial argentina).
*   Constitución → LUGAR → Nombre de una estación de subte (ferrocarril subterráneo).
*   La Boca → LUGAR → Nombre de un barrio.



---
# PARTE 3: Interfaces Interactivas con Gradio

Crearemos interfaces web interactivas para probar nuestros modelos.

In [24]:
import gradio as gr

def interfaz_ner_transformers(texto):
    """Interfaz para el modelo de Transformers"""
    if not texto.strip():
        return {"text": "Ingresa un texto para analizar", "entities": []}

    if not ner_pipeline:
        return {"text": "Modelo no disponible", "entities": []}

    # Procesar con Transformers
    entidades = ner_pipeline(texto)

    # Formatear para Gradio HighlightedText
    entidades_gradio = []
    for ent in entidades:
        entidades_gradio.append({
            "entity": ent["entity_group"],
            "word": ent["word"],
            "start": ent["start"],
            "end": ent["end"],
            "score": ent["score"]
        })

    return {"text": texto, "entities": entidades_gradio}

# Ejemplos para la interfaz
ejemplos_arg = [
    "Me llamo Juan Pérez y trabajo en el Banco Nación en Buenos Aires.",
    "Cristina Kirchner fue presidenta de Argentina y vive en Santa Cruz.",
    "River Plate jugará contra Boca Juniors en el estadio Monumental.",
    "Lionel Messi nació en Rosario y jugó en el Barcelona.",
    "La Universidad de La Plata es muy prestigiosa en Argentina."
]

# Crear interfaz
demo_transformers = gr.Interface(
    fn=interfaz_ner_transformers,
    inputs=[
        gr.Textbox(
            label="📝 Texto a analizar",
            placeholder="Escribe aquí tu texto en español...",
            lines=4
        )
    ],
    outputs=[
        gr.HighlightedText(
            label="🎯 Entidades Identificadas",
            show_legend=True
        )
    ],
    title="NER con Transformers - Español (de argentina)",
    description="""
    **Modelo:** `mrm8488/bert-spanish-cased-finetuned-ner`

    Identifica entidades nombradas en textos en español:
    - 🧑 **PER**: Personas
    - 🌍 **LOC**: Lugares
    - 🏢 **ORG**: Organizaciones
    - 📦 **MISC**: Misceláneo
    """,
    examples=ejemplos_arg,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

print("✅ Interfaz de Transformers creada")

✅ Interfaz de Transformers creada


In [25]:
# Interfaz para Gemini (solo si está disponible)
if cliente_gemini:
    def interfaz_ner_gemini(texto):
        """Interfaz para Gemini API"""
        if not texto.strip():
            return "Ingresa un texto para analizar"
        return analizar_entidades_gemini(texto)

    demo_gemini = gr.Interface(
        fn=interfaz_ner_gemini,
        inputs=[
            gr.Textbox(
                label="📝 Texto a analizar",
                placeholder="Escribe aquí tu texto en español...",
                lines=4
            )
        ],
        outputs=[
            gr.Textbox(
                label="🧠 Análisis de Gemini",
                lines=10
            )
        ],
        title="NER con Gemini - Análisis Detallado",
        description="""
        **Modelo:** Google Gemini 2.0 Flash

        Análisis avanzado de entidades nombradas con explicaciones contextuales
        optimizado para español argentino.
        """,
        examples=ejemplos_arg,
        allow_flagging="never",
        theme=gr.themes.Soft()
    )
    print("✅ Interfaz de Gemini creada")
else:
    print("⏭️  Interfaz de Gemini no creada (API Key no disponible)")

✅ Interfaz de Gemini creada


In [26]:
# Interfaz comparativa (solo si ambos están disponibles)
if ner_pipeline and cliente_gemini:
    def comparar_modelos(texto):
        """Compara resultados de ambos modelos"""
        if not texto.strip():
            return "Ingresa texto para comparar", "Ingresa texto para comparar"

        # Resultado Transformers
        entidades_tf = analizar_entidades_transformers(texto)
        resultado_tf = "TRANSFORMERS:\n\n"
        for ent in entidades_tf:
            resultado_tf += f"• {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})\n"

        # Resultado Gemini
        resultado_gemini = "GEMINI:\n\n" + analizar_entidades_gemini(texto)

        return resultado_tf, resultado_gemini

    demo_comparativo = gr.Interface(
        fn=comparar_modelos,
        inputs=[
            gr.Textbox(
                label="📝 Texto a comparar",
                placeholder="Ingresa texto para ver la comparación...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(label="Transformers", lines=8),
            gr.Textbox(label="Gemini", lines=8)
        ],
        title="⚔️ Comparación: Transformers vs Gemini",
        description="Compara los resultados de ambos enfoques lado a lado.",
        examples=[
            "Diego Maradona jugó en Boca Juniors y en el Napoli de Italia.",
            "El gobierno argentino anunció medidas desde Casa Rosada."
        ],
        allow_flagging="never"
    )
    print("✅ Interfaz comparativa creada")
else:
    print("⏭️  Interfaz comparativa no creada (requiere ambos modelos)")

✅ Interfaz comparativa creada


## 🚀 Lanzar Interfaces

Ejecuta las celdas siguientes para lanzar las interfaces interactivas:

In [27]:
# Lanzar interfaz de Transformers
if ner_pipeline:
    print("🚀 Lanzando interfaz de Transformers...")
    demo_transformers.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: modelo de Transformers no disponible")

🚀 Lanzando interfaz de Transformers...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a2af462efeb862fc90.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [28]:
# Lanzar interfaz de Gemini
if cliente_gemini:
    print("🚀 Lanzando interfaz de Gemini...")
    demo_gemini.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: API de Gemini no disponible")

🚀 Lanzando interfaz de Gemini...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bce8391c4ecfc9b0b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [29]:
# Lanzar interfaz comparativa
if ner_pipeline and cliente_gemini:
    print("🚀 Lanzando interfaz comparativa...")
    demo_comparativo.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: requiere ambos modelos disponibles")

🚀 Lanzando interfaz comparativa...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ed6bebcdbef5e1fc7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


---
# 🎓 EJERCICIOS

## 📝 Ejercicio 1: Personalización (BÁSICO)
1. Modifica los ejemplos para incluir más contexto argentino específico
2. Agrega 3 ejemplos nuevos con nombres de barrios porteños
3. Probá con texto de diferentes regiones de Argentina

## 🔧 Ejercicio 2: Análisis Comparativo (INTERMEDIO)
1. Crea una función que cuente cuántas entidades encuentra cada modelo
2. Implementa un sistema de métricas de tiempo de procesamiento
3. Analiza en qué casos cada modelo funciona mejor

## 🚀 Ejercicio 3: Extensiones Avanzadas (AVANZADO)
1. Implementa procesamiento en lote de múltiples textos
2. Crea una función de exportación de resultados a CSV
3. Desarrolla un sistema de filtrado por tipo de entidad

## 💡 Proyecto Integrador
Elegí una de estas aplicaciones y desarróllala:
- **Analizador de noticias argentinas**: Extrae personas y lugares de artículos
- **Procesador de CVs**: Identifica nombres, empresas y universidades
- **Análisis de redes sociales**: Detecta menciones de políticos y lugares

## 🤔 Preguntas de Reflexión
1. ¿Cuáles son las ventajas y desventajas de cada enfoque?
2. ¿En qué casos usarías un modelo local vs una API?
3. ¿Cómo evaluarías la precisión de los resultados?
4. ¿Qué consideraciones éticas debemos tener en cuenta?
5. ¿Cómo escalarías esta solución para procesar miles de documentos?

In [34]:
import gradio as gr # Importa la librería Gradio.

def interfaz_ner_transformers(texto):
    """
    Función de envoltorio para la interfaz de Gradio del modelo Transformers.
    Formatea la salida para ser compatible con gr.HighlightedText.
    """
    if not texto.strip(): # Verifica si el texto de entrada está vacío.
        return {"text": "Ingresa un texto para analizar", "entities": []} # Retorna mensaje si está vacío.

    if not ner_pipeline: # Verifica si el pipeline de Transformers está disponible.
        return {"text": "Modelo no disponible", "entities": []} # Retorna mensaje si no lo está.

    # Procesa el texto con el pipeline de Transformers.
    entidades = ner_pipeline(texto)

    # Formatea los resultados para el componente HighlightedText de Gradio.
    entidades_gradio = []
    for ent in entidades:
        entidades_gradio.append({
            "entity": ent["entity_group"], # La etiqueta de la entidad.
            "word": ent["word"],         # La palabra o frase de la entidad.
            "start": ent["start"],       # Posición de inicio en el texto.
            "end": ent["end"],           # Posición de fin en el texto.
            "score": ent["score"]        # Puntuación de confianza.
        })

    return {"text": texto, "entities": entidades_gradio} # Retorna el texto original y las entidades formateadas.

# Ejemplos para la interfaz de Gradio, con contexto argentino.
ejemplos_arg = [
    "Me llamo Juan Pérez y trabajo en el Banco Nación en Buenos Aires.",
    "Cristina Kirchner fue presidenta de Argentina y vive en Santa Cruz.",
    "River Plate jugará contra Boca Juniors en el estadio Monumental.",
    "Lionel Messi nació en Rosario y jugó en el Barcelona.",
    "La Universidad de La Plata es muy prestigiosa en Argentina.",
    # --- Ejercicio 1: Ejemplos nuevos con barrios porteños ---
    "El barrio de Palermo es famoso por sus parques y vida nocturna.",
    "Recoleta tiene edificios históricos y el Cementerio de la Recoleta.",
    "En Boedo se respira el tango y la tradición barrial."
]

# --- Interfaz para Transformers ---
demo_transformers = gr.Interface(
    fn=interfaz_ner_transformers, # La función que procesa la entrada.
    inputs=[
        gr.Textbox(
            label="📝 Texto a analizar",
            placeholder="Escribe aquí tu texto en español...",
            lines=4
        )
    ],
    outputs=[
        gr.HighlightedText( # Componente de Gradio que resalta entidades en el texto.
            label="🎯 Entidades Identificadas",
            show_legend=True # Muestra la leyenda de las etiquetas.
        )
    ],
    title="NER con Transformers - Español (de Argentina)", # Título de la interfaz.
    description="""
    **Modelo:** `mrm8488/bert-spanish-cased-finetuned-ner`

    Identifica entidades nombradas en textos en español:
    - 🧑 **PER**: Personas
    - 🌍 **LOC**: Lugares
    - 🏢 **ORG**: Organizaciones
    - 📦 **MISC**: Misceláneo
    """,
    examples=ejemplos_arg, # Ejemplos predefinidos.
    allow_flagging="never", # Deshabilita la opción de "flagging".
    theme=gr.themes.Soft() # Aplica un tema visual.
)

print("✅ Interfaz de Transformers creada")

# --- Interfaz para Gemini (solo si está disponible) ---
if cliente_gemini: # Solo se crea si la API Key de Gemini está configurada.
    def interfaz_ner_gemini(texto):
        """
        Función de envoltorio para la interfaz de Gradio del modelo Gemini.
        """
        if not texto.strip():
            return "Ingresa un texto para analizar"
        return analizar_entidades_gemini(texto) # Llama a la función de análisis con Gemini.

    demo_gemini = gr.Interface(
        fn=interfaz_ner_gemini,
        inputs=[
            gr.Textbox(
                label="📝 Texto a analizar",
                placeholder="Escribe aquí tu texto en español...",
                lines=4
            )
        ],
        outputs=[
            gr.Textbox( # Gemini retorna un texto plano con las entidades.
                label="🧠 Análisis de Gemini",
                lines=10
            )
        ],
        title="NER con Gemini - Análisis Detallado",
        description="""
        **Modelo:** Google Gemini 2.0 Flash

        Análisis avanzado de entidades nombradas con explicaciones contextuales
        optimizado para español argentino.
        """,
        examples=ejemplos_arg,
        allow_flagging="never",
        theme=gr.themes.Soft()
    )
    print("✅ Interfaz de Gemini creada")
else:
    print("⏭️  Interfaz de Gemini no creada (API Key no disponible)")

# --- Interfaz comparativa (solo si ambos están disponibles) ---
if ner_pipeline and cliente_gemini: # Solo se crea si ambos modelos están cargados.
    def comparar_modelos(texto):
        """
        Función para comparar los resultados de NER de ambos modelos lado a lado.
        """
        if not texto.strip():
            return "Ingresa texto para comparar", "Ingresa texto para comparar"

        # Formatea el resultado de Transformers para visualización de texto plano.
        entidades_tf = analizar_entidades_transformers(texto)
        resultado_tf = "TRANSFORMERS:\n\n"
        for ent in entidades_tf:
            resultado_tf += f"• {ent['texto']} → {ent['etiqueta']} (confianza: {ent['confianza']})\n"

        # Obtiene el resultado de Gemini.
        resultado_gemini = "GEMINI:\n\n" + analizar_entidades_gemini(texto)

        return resultado_tf, resultado_gemini # Retorna ambos resultados.

    demo_comparativo = gr.Interface(
        fn=comparar_modelos,
        inputs=[
            gr.Textbox(
                label="📝 Texto a comparar",
                placeholder="Ingresa texto para ver la comparación...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(label="Transformers", lines=8),
            gr.Textbox(label="Gemini", lines=8)
        ],
        title="⚔️ Comparación: Transformers vs Gemini",
        description="Compara los resultados de ambos enfoques lado a lado.",
        examples=[
            "Diego Maradona jugó en Boca Juniors y en el Napoli de Italia.",
            "El gobierno argentino anunció medidas desde Casa Rosada."
        ],
        allow_flagging="never"
    )
    print("✅ Interfaz comparativa creada")
else:
    print("⏭️  Interfaz comparativa no creada (requiere ambos modelos)")

✅ Interfaz de Transformers creada
✅ Interfaz de Gemini creada
✅ Interfaz comparativa creada


In [35]:
# Lanzar interfaz comparativa
if ner_pipeline and cliente_gemini:
    print("🚀 Lanzando interfaz comparativa...")
    demo_comparativo.launch(share=True, height=600)
else:
    print("❌ No se puede lanzar: requiere ambos modelos disponibles")

🚀 Lanzando interfaz comparativa...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a317f13436259d99d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Ejercicio 2)

In [31]:
import time # Importar el módulo time para medir el tiempo de ejecución.

print("\n--- EJERCICIO 2: Análisis Comparativo ---")

def contar_y_medir_entidades(texto):
    """
    Cuenta el número de entidades y mide el tiempo de procesamiento para cada modelo.
    """
    resultados_comparativos = {}

    # --- Transformers ---
    start_time_tf = time.time() # Inicia el cronómetro para Transformers.
    entidades_tf = analizar_entidades_transformers(texto)
    end_time_tf = time.time() # Detiene el cronómetro.
    tiempo_tf = end_time_tf - start_time_tf
    num_entidades_tf = len(entidades_tf)

    resultados_comparativos['transformers'] = {
        'num_entidades': num_entidades_tf,
        'tiempo_ms': round(tiempo_tf * 1000, 2), # Convierte a milisegundos.
        'entidades': entidades_tf # Guarda las entidades para análisis posterior.
    }

    # --- Gemini ---
    if cliente_gemini:
        start_time_gemini = time.time() # Inicia el cronómetro para Gemini.
        raw_gemini_output = analizar_entidades_gemini(texto)
        end_time_gemini = time.time() # Detiene el cronómetro.
        tiempo_gemini = end_time_gemini - start_time_gemini

        # Intentar contar entidades de Gemini (depende del formato de salida)
        # Esto es una aproximación, asumiendo una entidad por línea.
        num_entidades_gemini = len([line for line in raw_gemini_output.split('\n') if '→' in line])

        resultados_comparativos['gemini'] = {
            'num_entidades': num_entidades_gemini,
            'tiempo_ms': round(tiempo_gemini * 1000, 2),
            'raw_output': raw_gemini_output # Guarda la salida cruda para análisis.
        }
    else:
        resultados_comparativos['gemini'] = {
            'num_entidades': "N/A",
            'tiempo_ms': "N/A",
            'raw_output': "Cliente Gemini no disponible"
        }

    return resultados_comparativos

# --- Pruebas para el Análisis Comparativo ---
texto_prueba_1 = "El Dr. Facundo Manes visitó la Facultad de Medicina de la UBA en Recoleta."
texto_prueba_2 = "La empresa Tech Solutions Inc. inauguró su nueva sede en el Polo Tecnológico de la Ciudad de Córdoba."
texto_prueba_3 = "Lionel Messi ganó su octavo Balón de Oro en 2023. Juega para Inter Miami."

print(f"\n--- Analizando: '{texto_prueba_1}' ---")
comp_1 = contar_y_medir_entidades(texto_prueba_1)
print(f"  Transformers: {comp_1['transformers']['num_entidades']} entidades en {comp_1['transformers']['tiempo_ms']} ms")
print(f"  Gemini: {comp_1['gemini']['num_entidades']} entidades en {comp_1['gemini']['tiempo_ms']} ms")
print(f"  Entidades Transformers: {[ent['texto'] for ent in comp_1['transformers']['entidades']]}")
print(f"  Salida Gemini: \n{comp_1['gemini']['raw_output']}")

print(f"\n--- Analizando: '{texto_prueba_2}' ---")
comp_2 = contar_y_medir_entidades(texto_prueba_2)
print(f"  Transformers: {comp_2['transformers']['num_entidades']} entidades en {comp_2['transformers']['tiempo_ms']} ms")
print(f"  Gemini: {comp_2['gemini']['num_entidades']} entidades en {comp_2['gemini']['tiempo_ms']} ms")
print(f"  Entidades Transformers: {[ent['texto'] for ent in comp_2['transformers']['entidades']]}")
print(f"  Salida Gemini: \n{comp_2['gemini']['raw_output']}")

print(f"\n--- Analizando: '{texto_prueba_3}' ---")
comp_3 = contar_y_medir_entidades(texto_prueba_3)
print(f"  Transformers: {comp_3['transformers']['num_entidades']} entidades en {comp_3['transformers']['tiempo_ms']} ms")
print(f"  Gemini: {comp_3['gemini']['num_entidades']} entidades en {comp_3['gemini']['tiempo_ms']} ms")
print(f"  Entidades Transformers: {[ent['texto'] for ent in comp_3['transformers']['entidades']]}")
print(f"  Salida Gemini: \n{comp_3['gemini']['raw_output']}")

print("\n--- Análisis de Rendimiento y Casos (Ejercicio 2) ---")
print("Observaciones de las pruebas:")
print("- **Rendimiento (Tiempo)**: Generalmente, el modelo local de Transformers (si usa GPU) es **más rápido** que la llamada a la API de Gemini, que implica latencia de red. Para procesamiento en tiempo real o de alto volumen, Transformers es usualmente preferible.")
print("- **Cobertura y Precisión**: ")
print("  - **Transformers**: Excelente para las categorías estándar (PER, LOC, ORG). Puede ser muy preciso en entidades conocidas si fueron vistas en su entrenamiento.")
print("  - **Gemini**: Su fuerza radica en la **flexibilidad y contextualización**. Puede identificar entidades 'MISC' de forma más robusta o incluso entidades que no encajan perfectamente en las categorías predefinidas de un modelo NER tradicional. Además, su explicación ('BREVE EXPLICACIÓN') es un valor agregado.")
print("  - **Casos de mejora**: En textos con lenguaje coloquial, neologismos o entidades muy específicas/nuevas que no estén en el dataset de entrenamiento del modelo de Transformers, Gemini (como LLM) podría tener una mejor capacidad de inferencia debido a su conocimiento general del mundo.")
print("  - **Inconsistencias**: Transformers puede a veces dividir entidades multi-palabra de forma incorrecta si `aggregation_strategy` no es perfecto o el modelo no fue entrenado para ese patrón específico. Gemini, al ser generativo, puede a veces 'inventar' una explicación o clasificar de forma inesperada si el prompt no es lo suficientemente restrictivo o el contexto es ambiguo.")
print("-" * 50)


--- EJERCICIO 2: Análisis Comparativo ---

--- Analizando: 'El Dr. Facundo Manes visitó la Facultad de Medicina de la UBA en Recoleta.' ---
  Transformers: 5 entidades en 270.66 ms
  Gemini: 4 entidades en 1107.98 ms
  Entidades Transformers: ['Dr', '. Facundo Manes', 'Facultad de Medicina', 'UBA', 'Recoleta']
  Salida Gemini: 
*   Dr. Facundo Manes → PERSONA → Nombre de una persona, precedido por un título (Dr.).
*   Facultad de Medicina → ORGANIZACIÓN → Nombre de una facultad.
*   UBA → ORGANIZACIÓN → Siglas de la Universidad de Buenos Aires.
*   Recoleta → LUGAR → Nombre de un barrio en Buenos Aires.


--- Analizando: 'La empresa Tech Solutions Inc. inauguró su nueva sede en el Polo Tecnológico de la Ciudad de Córdoba.' ---
  Transformers: 2 entidades en 215.49 ms
  Gemini: 3 entidades en 644.09 ms
  Entidades Transformers: ['Tech Solutions Inc.', 'Polo Tecnológico de la Ciudad de Córdoba']
  Salida Gemini: 
*   Tech Solutions Inc. → ORGANIZACIÓN → Nombre de la empresa
*   Polo Tec

Ejercicio 3

In [32]:
import pandas as pd # Importar pandas para exportar a CSV y manipulación de datos.

print("\n--- EJERCICIO 3: Extensiones Avanzadas ---")

# --- 3.1 Procesamiento en Lote ---
def procesar_lote_transformers(lista_textos):
    """
    Procesa una lista de textos con el pipeline de Transformers.
    """
    if not ner_pipeline:
        return []

    print(f"Procesando {len(lista_textos)} textos con Transformers...")
    resultados_lote = []
    for i, texto in enumerate(lista_textos):
        entidades = analizar_entidades_transformers(texto)
        resultados_lote.append({'id_texto': i, 'texto_original': texto, 'entidades': entidades})
    return resultados_lote

def procesar_lote_gemini(lista_textos):
    """
    Procesa una lista de textos con Gemini.
    """
    if not cliente_gemini:
        return []

    print(f"Procesando {len(lista_textos)} textos con Gemini...")
    resultados_lote = []
    for i, texto in enumerate(lista_textos):
        respuesta_gemini = analizar_entidades_gemini(texto)
        resultados_lote.append({'id_texto': i, 'texto_original': texto, 'gemini_raw_output': respuesta_gemini})
    return resultados_lote

# Ejemplo de uso en lote
textos_para_lote = [
    "Juan trabaja en Aerolíneas Argentinas y vive en Caballito.",
    "El Museo Nacional de Bellas Artes está en Recoleta, Buenos Aires.",
    "La AFA organizó un torneo en el predio de Ezeiza.",
    "Mercedes Sosa fue una cantante folclórica de Tucumán."
]

print("\n--- Procesamiento en Lote (Transformers) ---")
lote_transformers = procesar_lote_transformers(textos_para_lote)
for res in lote_transformers:
    print(f"Texto ID {res['id_texto']}: {len(res['entidades'])} entidades")
    # print(res['entidades']) # Descomentar para ver todas las entidades

if cliente_gemini:
    print("\n--- Procesamiento en Lote (Gemini) ---")
    lote_gemini = procesar_lote_gemini(textos_para_lote)
    for res in lote_gemini:
        print(f"Texto ID {res['id_texto']}: \n{res['gemini_raw_output']}\n---")

# --- 3.2 Función de Exportación a CSV ---
def exportar_a_csv_transformers(resultados_lote_transformers, nombre_archivo="entidades_transformers.csv"):
    """
    Exporta los resultados de NER de Transformers a un archivo CSV.
    Cada fila representa una entidad.
    """
    registros = []
    for res_texto in resultados_lote_transformers:
        for ent in res_texto['entidades']:
            registros.append({
                'id_texto': res_texto['id_texto'],
                'texto_original': res_texto['texto_original'],
                'entidad_texto': ent['texto'],
                'etiqueta': ent['etiqueta'],
                'confianza': ent['confianza'],
                'posicion_inicio': ent['posicion'][0],
                'posicion_fin': ent['posicion'][1]
            })

    df = pd.DataFrame(registros)
    df.to_csv(nombre_archivo, index=False)
    print(f"Resultados de Transformers exportados a '{nombre_archivo}'")

def exportar_a_csv_gemini(resultados_lote_gemini, nombre_archivo="entidades_gemini.csv"):
    """
    Exporta los resultados crudos de Gemini a un archivo CSV.
    Cada fila es la salida completa de Gemini para un texto.
    """
    registros = []
    for res_texto in resultados_lote_gemini:
        registros.append({
            'id_texto': res_texto['id_texto'],
            'texto_original': res_texto['texto_original'],
            'gemini_raw_output': res_texto['gemini_raw_output']
        })
    df = pd.DataFrame(registros)
    df.to_csv(nombre_archivo, index=False)
    print(f"Resultados de Gemini (salida raw) exportados a '{nombre_archivo}'")

print("\n--- Exportando resultados a CSV ---")
exportar_a_csv_transformers(lote_transformers)
if cliente_gemini:
    exportar_a_csv_gemini(lote_gemini)

# --- 3.3 Sistema de Filtrado por Tipo de Entidad ---
def filtrar_entidades_transformers(entidades, tipo_entidad=None):
    """
    Filtra una lista de entidades de Transformers por tipo de entidad.
    """
    if tipo_entidad is None:
        return entidades # Si no se especifica tipo, retorna todas.

    entidades_filtradas = [ent for ent in entidades if ent['etiqueta'].lower() == tipo_entidad.lower()]
    return entidades_filtradas

# Ejemplo de uso del filtro
texto_filtrar = "El presidente se reunió con la Dra. López en la Casa de Gobierno de Salta."
entidades_texto_filtrar = analizar_entidades_transformers(texto_filtrar)

print(f"\n--- Entidades de '{texto_filtrar}' ---")
for ent in entidades_texto_filtrar:
    print(f"  • {ent['texto']} → {ent['etiqueta']}")

print("\n--- Entidades filtradas (solo PERSONA) ---")
personas = filtrar_entidades_transformers(entidades_texto_filtrar, tipo_entidad='PER')
for p in personas:
    print(f"  • {p['texto']}")

print("\n--- Entidades filtradas (solo LUGAR) ---")
lugares = filtrar_entidades_transformers(entidades_texto_filtrar, tipo_entidad='LOC')
for l in lugares:
    print(f"  • {l['texto']}")
print("-" * 50)


--- EJERCICIO 3: Extensiones Avanzadas ---

--- Procesamiento en Lote (Transformers) ---
Procesando 4 textos con Transformers...
Texto ID 0: 3 entidades
Texto ID 1: 3 entidades
Texto ID 2: 3 entidades
Texto ID 3: 2 entidades

--- Procesamiento en Lote (Gemini) ---
Procesando 4 textos con Gemini...
Texto ID 0: 
Aquí está la extracción de entidades nombradas del texto proporcionado, clasificadas según las categorías indicadas:

*   **Juan** → PERSONA → Nombre de una persona.
*   **Aerolíneas Argentinas** → ORGANIZACIÓN → Nombre de una empresa de transporte aéreo.
*   **Caballito** → LUGAR → Nombre de un barrio de la Ciudad de Buenos Aires.

---
Texto ID 1: 
Aquí está la extracción y clasificación de las entidades nombradas del texto proporcionado:

*   Museo Nacional de Bellas Artes → ORGANIZACIÓN → Nombre de un museo.
*   Recoleta → LUGAR → Nombre de un barrio en Buenos Aires.
*   Buenos Aires → LUGAR → Nombre de una ciudad.

---
Texto ID 2: 
*   AFA → ORGANIZACIÓN → Asociación del Fút

In [30]:
# 📝 ESPACIO PARA TUS EJERCICIOS
# Usa esta celda para experimentar y desarrollar tus soluciones

# Ejemplo: Función para contar entidades por tipo
def contar_entidades_por_tipo(texto):
    """Cuenta entidades por categoría usando Transformers"""
    if not ner_pipeline:
        return {}

    entidades = ner_pipeline(texto)
    conteo = {}

    for ent in entidades:
        tipo = ent['entity_group']
        if tipo in conteo:
            conteo[tipo] += 1
        else:
            conteo[tipo] = 1

    return conteo

# Probar la función
texto_prueba = "Juan Pérez trabaja en Google Argentina en Buenos Aires con María López."
print("📊 Conteo de entidades:")
print(contar_entidades_por_tipo(texto_prueba))

# TODO: Agrega aquí tus propias funciones y experimentos

📊 Conteo de entidades:
{'PER': 2, 'ORG': 1, 'LOC': 1}


---
# 🎯 Conclusión

¡Felicitaciones! Completaste el ejercicio de Reconocimiento de Entidades Nombradas.

## 📚 Lo que aprendiste:
- ✅ Implementar NER con modelos pre-entrenados
- ✅ Usar APIs de IA generativa para tareas de PLN
- ✅ Crear interfaces interactivas con Gradio
- ✅ Comparar diferentes enfoques de NER

## 🔄 Próximos pasos:
1. Experimentá con otros modelos de Hugging Face
2. Probá con textos de diferentes dominios
3. Implementa tu proyecto integrador
4. Compartí tus resultados con la clase

## 📖 Recursos adicionales:
- [Hugging Face Models](https://huggingface.co/models?pipeline_tag=token-classification&language=es)
- [Gradio Documentation](https://gradio.app/docs/)
- [Google AI Studio](https://ai.google.dev/)

---
**¡Éxito en tu trabajo integrador!** 🎓🚀